In [ ]:
username = "admin"
password = "bodyguard"

database = "homemonitor"
retention_policy = "autogen"

bucket = f"{database}/{retention_policy}"

time_window = 600

In [1]:
trend_config = [
    {
        "title": "Puissance apparente (W)",
        "measurement": "teleinfo",
        "field": "PAPP",
        "refresh_rate": 5,
    },
    {
        "title": "Courant instantané (A)",
        "measurement": "teleinfo",
        "field": "IINST",
        "refresh_rate": 10,
    },
    {
        "title": "Puissance apparente (W)",
        "measurement": "teleinfo",
        "field": "PAPP",
        "refresh_rate": 5,
    },
    {
        "title": "Courant instantané (A)",
        "measurement": "teleinfo",
        "field": "IINST",
        "refresh_rate": 10,
    },
]

temperature_thresholds = {18: 'blue',
                  22: 'green',
                  26: 'orange',
                  50: 'red'}

humidity_thresholds = {40: 'red',
                  60: 'green',
                  100: 'red'}

number_config = [
    {'name': "Température",
     "measurement": "sensorhub",
     "field": "TempOffchip",
     "refresh_rate": 60,
     "thresholds": temperature_thresholds,
     "string_format": '{value:.1f}°C'},
    {'name': "Humidité",
    "measurement": "sensorhub",
    "field": "Humidity",
    "refresh_rate": 60,
     "thresholds": humidity_thresholds,
     "string_format": '{value:0.0f}%'},
    {'name': "Pression",
    "measurement": "sensorhub",
    "field": "Pressure",
    "refresh_rate": 60,
     "string_format": '{value:.4f} Bar'},
]


In [8]:
number_config[0].get('threshold')

In [ ]:
import pandas as pd
from influxdb_client import InfluxDBClient

import panel as pn

pn.extension(sizing_mode = 'stretch')

In [ ]:
def query_data(query_api, bucket: str, measurement: str, field: str, time_range: int):
    
    query = f'''from(bucket: "{bucket}")
                |> range(start: -{time_range}s, stop: now())
                |> filter(fn: (r) => r._measurement == "{measurement}")
                |> filter(fn: (r) => r._field == "{field}")
                |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
                |> keep(columns: ["_time", "{field}"])'''
    
    df = query_api.query_data_frame(query, data_frame_index=['_time'])
    df = df.drop(columns=['result', 'table']).reset_index()
    
    return df

In [ ]:
def query_mean_value(query_api, bucket: str, measurement: str, field: str, time_range: int):
    
    query = f'''from(bucket: "{bucket}")
                |> range(start: -{time_range}s, stop: now())
                |> filter(fn: (r) => r._measurement == "{measurement}")
                |> filter(fn: (r) => r._field == "{field}")
                |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
                |> keep(columns: ["_time", "{field}"])'''
    
    df = query_api.query_data_frame(query, data_frame_index=['_time'])
    
    return df[field].mean()

In [ ]:
def create_trend(query_api, bucket, measurement, field, time_window, refresh_rate, title=''):
    
    df_init = query_data(query_api, bucket, measurement, field, time_window)
    window_size = df_init.shape[0]
    
    trend = pn.indicators.Trend(title=title, data=df_init, plot_x='_time', plot_y=field, width=400, height=300, plot_type='area')

    def update_plot():
        trend.stream(query_data(query_api, bucket, measurement, field, refresh_rate), rollover=window_size)

    pn.state.add_periodic_callback(update_plot, refresh_rate * 1000)

    return trend

In [ ]:
def create_number(query_api,  bucket: str, measurement: str, field: str, refresh_rate: int, name='', string_format='{value:.1f}',thresholds=None):
    
    if thresholds is not None: 
        thresholds = list(thresholds.items())
    
    mean_value = query_mean_value(query_api, bucket, measurement, field, refresh_rate)
    number = pn.indicators.Number(name=name, value=mean_value, format=string_format, width=400, height=200,
                                  colors=thresholds)
    
    def update_value():
        number.value = query_mean_value(query_api, bucket, measurement, field, refresh_rate)
    
    pn.state.add_periodic_callback(update_value, refresh_rate * 1000)
    
    return number

In [ ]:

client = InfluxDBClient(
    url="http://192.168.1.16:8086", token=f"{username}:{password}", org="-"
)
    
query_api = client.query_api()
layout1 = pn.Row(*[create_trend(query_api, bucket, time_window=time_window, **config) for config in trend_config])
layout2 = pn.Row(*[create_number(query_api,  bucket, **config) for config in number_config])

In [ ]:

# number = pn.indicators.Number(
#     name='Failure Rate', value=72, format='{value}%',
#     colors=[(33, 'green'), (66, 'gold'), (100, 'red')]
# )

In [ ]:
template = pn.template.FastListTemplate(title="Home monitoring", main=[ pn.Card(layout1, title='Electrictité'),
                                                                       pn.Card(layout2, title='Capteurs cuisine')], main_layout='')
template.servable()